In [25]:
#-*- coding: utf-8 -*-

import logging
from gensim.models import Word2Vec
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from __future__ import print_function
import pprint


plt.style.use(u'grayscale')
plt.style.available
%matplotlib inline  


In [2]:
# review data load 
review_file = './resource/mr.150813'
review_ds = pd.read_csv(review_file,sep='\t',index_col=None, header=None, \
            names=['idx','title','review','rating','source','timestamp'])


/Users/goodvc/anaconda/lib/python2.7/site-packages/pandas/io/parsers.py:1139: DtypeWarning: Columns (3,5) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [28]:
review_ds.head(2)

,idx,title,review,rating,source,timestamp
0,1,10억,몇년동안 본거중에 가장 최악이다 한국영화여서 챙피하다...,1,Naver movie,1437446025
1,2,10억,와. .진심 그 피디 죽일수있었는데 못죽이고ㅡㅡ해튼 주연이라는 신민아 연기는 아주 ...,1,Naver movie,1437446025


In [52]:
## list, dict 타입 내의 한글 출력을 위한 클래스 
## 여기에서 가져옴 (참고) : http://egloos.zum.com/mcchae/v/11076302 

import pprint
class MyPrettyPrinter(pprint.PrettyPrinter):
    def format(self, _object, context, maxlevels, level):
        if isinstance(_object, unicode):
            return "'%s'" % _object.encode('utf8'), True, False
        elif isinstance(_object, str):
            _object = unicode(_object,'utf8')
            return "'%s'" % _object.encode('utf8'), True, False
        return pprint.PrettyPrinter.format(self, _object, context, maxlevels, level)

def pp(obj):
    MyPrettyPrinter().pprint(obj)

---
## 영화 리뷰 데이터셋 트레이닝

In [46]:
# Nan value drop 
review_ds = review_ds.dropna()

# corpus generaion
sentances = []
for idx, row in review_ds.iterrows():
    # words = nonLetterTokenizer(str(row['review']))
    words = str(row['review']).split()
    # words.append( row['title'] )
    sentances.append( words )

In [64]:
print('total reviews', len(sentances))

total reviews 697435


In [65]:
pp(sentances[:2])

[['몇년동안',
  '본거중에',
  '가장',
  '최악이다',
  '한국영화여서',
  '챙피하다...'],
 ['와.',
  '.진심',
  '그',
  '피디',
  '죽일수있었는데',
  '못죽이고ㅡㅡ해튼',
  '주연이라는',
  '신민아',
  '연기는',
  '아주',
  '발연기!!',
  '.',
  '.',
  '신민아',
  '와',
  '대박이다',
  '심하게',
  '못한다']]


In [7]:
# word2vec training
model = Word2Vec(sentances, window=5, min_count=5, size=100)

# print matrix size 
print (model.syn0.shape)

(113367, 100)


In [8]:
model.save('./resource/moview_review_ds.model')

In [19]:
print (model['이정재'])

[-0.67047834 -0.51212126  0.31987262  0.30438229 -0.72509873 -1.29164243
  0.29205215 -0.60837358  0.03511703  0.85490364  1.15798008 -1.43304574
 -1.16789186  0.1444733   0.65131485 -2.26791978  0.34301984  0.6875357
 -1.20729697  0.45071012  0.48766235 -0.96822786  0.34821603  1.52888072
 -0.36123639  0.71544635  0.58807594  0.89164084  0.49277171 -0.04102636
 -1.03266406 -0.3597382  -1.36048067  0.37535775 -0.16482817 -1.56930912
 -0.79351777 -0.11401723  1.39298224  0.72664005  0.1866966  -0.95610803
 -0.5053364  -0.71753675 -0.01030337 -1.33528054  1.16171587 -0.74348259
 -0.64804882  0.04825107 -1.21049678 -0.77956378 -0.06175468  1.21322596
 -1.52820933 -1.51002514  0.8124941   0.71251398  0.91616482  1.24890971
 -1.5778265   0.24737386  0.2455882  -1.46497047 -0.08189955 -0.51065743
  2.27265072 -0.06629444  1.41176713  0.79846281 -0.82127374  0.95520878
  0.30863282 -0.15272142  0.8401857  -0.32685298 -0.66309249  0.80250353
 -0.92467946  0.17398819 -0.03312703  0.15617493 -0.

In [66]:
pp( model.most_similar(positive=['이정재'], topn=10))

[('조진웅', 0.941116452217102),
 ('전지현', 0.9344793558120728),
 ('하정우', 0.9260144829750061),
 ('조정석', 0.9184736013412476),
 ('고수', 0.9184537529945374),
 ('오달수', 0.9164066314697266),
 ('한석규', 0.9155168533325195),
 ('정우성', 0.9113757610321045),
 ('마동석', 0.9107865691184998),
 ('악역', 0.9103112816810608)]


In [67]:
pp(model.most_similar(positive=['암살'], topn=10))

[('군도', 0.9122183918952942),
 ('명량', 0.8987618684768677),
 ('용의자', 0.8760471343994141),
 ('해적', 0.8748217225074768),
 ('해무', 0.8721737265586853),
 ('설국열차', 0.8642172813415527),
 ('베테랑', 0.8638518452644348),
 ('헝거게임', 0.8601628541946411),
 ('겨울왕국', 0.8526073694229126),
 ('역린', 0.8522467017173767)]


In [68]:
pp(model.most_similar(positive=['이정재','관상'], negative=['암살'], topn=10))

[('재발견..', 0.831844687461853),
 ('정재영,', 0.8292697668075562),
 ('윤여정', 0.8232628107070923),
 ('이정재와', 0.8195985555648804),
 ('변신이', 0.809780478477478),
 ('뿜어내는', 0.8090252876281738),
 ('백윤식', 0.807701826095581),
 ('전혜진,', 0.8065884709358215),
 ('악역과', 0.8054690361022949),
 ('수양대군', 0.8053158521652222)]


In [69]:
pp(model.most_similar(positive=['암살','베테랑'], negative=['이정재'], topn=10))

[('명량', 0.8765701055526733),
 ('국제시장', 0.8422200679779053),
 ('인터스텔라',
  0.8336617946624756),
 ('군도', 0.8252524733543396),
 ('4D', 0.8175791501998901),
 ('설국열차', 0.8139926195144653),
 ('해적', 0.8111826181411743),
 ('헝거게임', 0.8075121641159058),
 ('3d', 0.7999213933944702),
 ('조조', 0.7932279706001282)]


In [70]:
pp(model.doesnt_match('이정재 하정우 전지현 백윤식'.split()))

'백윤식'


## 단어 클러스터링하기 

In [71]:
## 클러스터수 결정
import math 

k = math.ceil(math.sqrt(len(model.index2word)/2)) 
print ( "cluster 수(k) = " , k)

cluster 수(k) =  239.0


In [72]:
vec = model.syn0
index2word = model.index2word

In [73]:
vec.shape

(113367, 100)

In [74]:
import mykmeans as kmeans
(centres, index2cluster, dist) = kmeans.kmeanssample( vec, k, nsample=0, maxiter=20, metric='cosine',delta=0.000001 )

kmeans: X (2390, 100)  centres (239, 100)  delta=1e-06  maxiter=20  metric=cosine
('kmeans: 20 iterations  cluster sizes:', array([35, 19,  8,  0, 11,  1,  1,  1,  4,  1, 15, 14, 60,  8,  5, 17,  7,
        0,  2,  2,  0,  7,  0,  0,  1, 36,  0,  3, 37,  4,  1,  3,  1,  2,
       10,  4, 20,  5,  0,  2, 13,  4,  4,  1,  1,  1,  2, 14,  8,  2,  3,
        1, 32,  0,  0,  1,  1,  5, 23, 11,  3, 11, 31, 22, 25, 17,  5,  1,
        1, 13,  1, 11, 12,  4,  0,  0,  1,  4, 50,  1, 25,  1, 24,  6, 34,
       17,  7, 31,  1, 24,  1,  7,  1,  1,  3,  7,  8,  2,  1,  0,  3,  9,
        2,  1,  1,  5,  4, 21,  5, 14,  4, 85,  1,  5, 14, 36, 21,  1,  1,
        2,  4,  2,  7, 10, 55, 59, 28,  7,  2,  2,  8,  6,  4,  0, 11,  6,
        1,  1,  1,  2,  8,  7,  5, 10,  2,  4, 39,  6,  2,  2, 23,  0, 19,
        9,  2,  3,  5,  1,  2,  2, 11, 15,  2, 32,  5, 60,  1,  7,  1, 12,
        1, 42,  0,  4,  8, 30, 16,  1, 17, 13, 39, 17,  5,  1,  7,  0, 46,
       11,  8,  2,  9, 76,  5,  2,  8, 13,  3,  1, 

In [76]:
info = [[a,b] for a,b in zip(index2cluster, dist)]
clustered = pd.DataFrame(info, index=index2word, columns=['cid','distance'])
clustered.head(3)

,cid,distance
영화,21,0.320669
너무,43,0.422360
정말,33,0.210787


In [77]:
grouped = clustered.groupby(['cid']).groups

In [84]:
pp(grouped[4])

['지루하지',
 '맞지',
 '지루하지도',
 '어울리지',
 '놓치지',
 '생각지도',
 '뻔하지',
 '질리지',
 '같지',
 '어색하지',
 '잃지',
 '빠지지',
 '과하지',
 '믿기지',
 '무겁지',
 '예상치',
 '유치하지',
 '자극적이지',
 '가볍지',
 '못지',
 '무섭지',
 '억지스럽지',
 '어렵지',
 '무겁지도',
 '가볍지도',
 '돈아깝지',
 '길지',
 '흔치',
 '잔인하지',
 '부족하지',
 '하지도',
 '과하지도',
 '생각지',
 '조금도',
 '흥미롭지',
 '끊이지',
 '화려하지',
 '촌스럽지',
 '끊이질',
 '나오지도',
 '웃기지',
 '진부하지',
 '알려지지',
 '식상하지',
 '신선하지',
 '실망하지',
 '야하지',
 '지겹지',
 '이해되지',
 '가볍지만',
 '평범하지',
 '떨어지지',
 '졸지',
 '뒤지지',
 '같지도',
 '새롭지',
 '보고싶지',
 '흔하지',
 '쉬지',
 '예상하지',
 '잔인하지도',
 '하진',
 '이어지지',
 '지루하지가',
 '불편하지',
 '있지도',
 '질리지가',
 '보이지도',
 '슬프지',
 '밀리지',
 '답지',
 '슬프지도',
 '공감되지',
 '과장되지',
 '가볍지만은',
 '아무렇지도',
 '현실적이지',
 '화려하진',
 '완벽하진',
 '되지도',
 '치우치지',
 '만만치',
 '뜻하지',
 '드러나지',
 '공감가지',
 '세련되지',
 '적지',
 '감동적이지도',
 '부담스럽지',
 '닿지',
 '익숙하지',
 '생각치도',
 '늙지도',
 '심심하지',
 '다가오지',
 '개운치',
 '심각하지',
 '오버하지',
 '특별하지',
 '생각치',
 '어울리지도',
 '명확하지',
 '복잡하지',
 '개운하지',
 '모자라지도',
 '아쉽지',
 '영화같지',
 '유치하지도',
 '사리지',
 '완벽하지',
 '기억나지',
 '같지가',
 '낯설지',
 '그렇지도',
 '꿀리지',
 '화려하지도',
 '

In [131]:
def word2cluster_members(w):
    (cid,sim) = clustered.loc[w]
    return (sim, clustered[clustered.cid==cid].sort(['distance'], ascending=False).index.tolist())

In [134]:
pp(word2cluster_members('이정재'))

(0.070089358611433838,
 ['같구.',
  '경치',
  '형편없고,',
  '몰입',
  '어설픈거',
  '둘다',
  '띄네요',
  '이뻐서',
  '멋있게',
  '예뻐서',
  '매력',
  '제이슨',
  '제옷을',
  '정평이',
  '카리스마',
  '액션씬',
  'ㄷ',
  '존재감',
  '주인공도',
  '송강호',
  '연기너무',
  '하정우가',
  '주인공들',
  '액션신',
  '악역도',
  '아저씨',
  '굿.',
  '발연기',
  '연기력에만',
  '더빙',
  '전지현이',
  '이하늬도',
  '강동원.',
  '김남길이',
  '멋있고',
  '포스가',
  '박유천씨',
  '쩔고',
  '여배우',
  '잘생긴',
  '요한슨',
  '목소리도',
  '연기랑',
  '이쁘다',
  '탁월함',
  '몸매도',
  '전도연',
  '이쁘게나옴',
  '매력있고',
  '연기잘하고',
  '이병헌',
  '외모도',
  '찰지다.',
  '사투리',
  '멋있다',
  '유해진이',
  '강동원도',
  '한효주',
  '탑',
  '하정우나',
  '아역',
  '전지현은',
  '김우빈',
  '배우님',
  '여주인공',
  '멜리사',
  '사이프리드의',
  '역',
  '여배우도',
  '여주인공도',
  '변신',
  '이정재는',
  '명불허전임',
  '이정재,',
  '최민식',
  '조연들',
  '컴버배치',
  '박보영',
  '다니엘',
  '조연들도',
  '하정우',
  '여주',
  '황정민',
  '니콜라스',
  '여배우들',
  '유아인',
  '샤를리즈',
  '하지원',
  '얼굴도',
  '그외',
  '목소리',
  '몸매가',
  '언니',
  '하정우는',
  '잘생기고',
  '김수현',
  '주지훈',
  '여주도',
  '전지현',
  '전도연,',
  '이쁨',
  '황정민,',
  '케미',
  '몸매',
  '남주',
  '이승기',